In [34]:
import pygame
import os
import random

# Initializing Pygame
pygame.init()

# Setting up the display of the game
width, height = 1157, 750
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Rummikub Game")

# name of the folder conatining all the cards of the game
card_folder = "generated_cards"

# defining the function to call the images from the folder
def load_card_images():
    card_images = []
    for file_name in os.listdir(card_folder):
        if file_name.endswith(".png"):
            file_path = os.path.join(card_folder, file_name)
            card_image = pygame.image.load(file_path)
            card_image = pygame.transform.scale(card_image, (50, 70))  # size of the cards
            card_images.append(card_image)
    return card_images

# distribute the cards and keep the other cards in the centre
def distribute_cards(all_cards):
    random.shuffle(all_cards)
    player1_cards = all_cards[:14]
    player2_cards = all_cards[14:28]
    dealer_cards = all_cards[28:]
    return player1_cards, player2_cards, dealer_cards

def display_hand(cards, x, y, rows=2, cards_per_row=7, player_name="", player_icon=""):
    gap_x = 10  # Gap between the cards
    gap_y = 15  # Gap between the card rows
    icon_gap = 20  # Cap between the icon and the cards

    if player_icon:
        icon = pygame.image.load(player_icon)
        icon = pygame.transform.scale(icon, (40, 40))  # To adust the size of the icon
        screen.blit(icon, (x, y))  # Displaying the icon

        # Display player name text below the icon
        font = pygame.font.Font(None, 24)
        text = font.render(player_name, True, (255, 255, 255))  # displaying text below the icon in white color
        text_rect = text.get_rect()
        text_rect.topleft = (x + icon.get_width() // 2 - text.get_width() // 2, y + icon.get_height() + 5)
        screen.blit(text, text_rect)

        x += icon.get_width() + icon_gap  # Update x position for cards

    for row in range(rows):
        for col in range(cards_per_row):
            index = row * cards_per_row + col
            if index < len(cards):
                card = cards[index]
                card_width = card.get_width()
                card_height = card.get_height()
                screen.blit(card, (x + col * (card_width + gap_x), y + row * (card_height + gap_y)))

def display_remaining(cards, x, y):
    stack_gap = 0  # to keep all the cards as a stack

    for i, card in enumerate(cards):
        screen.blit(card, (x, y + i * stack_gap))

def main():
    clock = pygame.time.Clock()
    all_cards = load_card_images()
    player1_cards, player2_cards, dealer_cards = distribute_cards(all_cards)
    
    # Load player icons
    player1_icon = "player_1.jpg"  # Player 1 icon Location
    player2_icon = "Player_2.jpg"  # Player 2 icon location

    # background image loading from the folder
    background_image = pygame.image.load("background_image_welcome screen.jpg") # path to the background image

    selected_card = None  # Track selected card for placement

    player1_selected_cards = []  # Store selected cards for Player 1 in a separate row
    player2_selected_cards = []  # Store selected cards for Player 2 in a separate row

    running = True 
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                # Check if the mouse click is on the area above Player 1's cards
                if 30 <= mouse_x <= 200 + 7 * 65 and 500 <= mouse_y <= 700:
                    selected_index = (mouse_x - 50) // 65
                    if selected_index < len(player1_cards):  # Adjusted condition
                        selected_card = player1_cards.pop(selected_index)
                        # Add the selected card to Player 1's separate row for display
                        player1_selected_cards.append(selected_card)
                # Check if the mouse click is on the area above Player 2's cards
                elif 30 <= mouse_x <= 200 + 7 * 65 and 50 <= mouse_y <= 250:
                    selected_index = (mouse_x - 50) // 65
                    if selected_index < len(player2_cards):  # Adjusted condition
                        selected_card = player2_cards.pop(selected_index)
                        # Add the selected card to Player 2's separate row for display
                        player2_selected_cards.append(selected_card)
                # Check if the mouse click is on the dealer's card stack area
                elif width - 150 <= mouse_x <= width - 70 and height // 2 <= mouse_y <= height // 2 + 80:
                    if dealer_cards:  # Ensure there are cards in the dealer's stack
                        drawn_card = dealer_cards.pop()
                        # Add the drawn card to Player 1 or Player 2's row for display
                        if len(player1_cards) < 14:  # Player 1's turn
                            player1_cards.append(drawn_card)
                        elif len(player2_cards) < 14:  # Player 2's turn
                            player2_cards.append(drawn_card)

        # calling background image
        screen.blit(background_image, (0, 0))

        # Display Player 1's selected cards in a separate row
        display_hand(player1_selected_cards, x=50, y=400, rows=1, cards_per_row=len(player1_selected_cards),
                     player_name="Player 1's Selected Cards", player_icon="")  # Adjust the position and size as needed

        # Display Player 2's selected cards in a separate row
        display_hand(player2_selected_cards, x=50, y=300, rows=1, cards_per_row=len(player2_selected_cards),
                     player_name="Player 2's Selected Cards", player_icon="")  # Adjust the position and size as needed

        # Player 1 cards with Player 1 icon
        display_hand(player1_cards, x=50, y=500, player_name="Player 1", player_icon=player1_icon)

        # Player 2 cards with Player 2 icon
        display_hand(player2_cards, x=50, y=50, player_name="Player 2", player_icon=player2_icon)

        # Remaining cards as a stack
        display_remaining(dealer_cards, x=width - 150, y=height // 2)

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

main()

